### Generate the equations of the robot's momentum

In [1]:
import symforce as sf
sf.set_epsilon_to_symbol()
sf.set_symbolic_api('sympy')
sf.set_log_level('warning')
import symforce.symbolic as sb
import numpy as np
# exoskeleton's parameter

# link length
foot_l_e = 0.24
foot_h_e = 0.114
calf_l_e = 0.4505
thigh_l_e = 0.438

# mass
m_calf_e = 1
m_thigh_e = 1.2
m_backpack = 10.20583
m_foot_e = 0.8242665

# com position
# f: front, r: rear
f_calf_com_e = sb.Vector2(0.2557,-0.0236)# calculated from solidwork 
r_calf_com_e = sb.Vector2(0.1968,0.0392)
f_thigh_com_e =sb.Vector2(0.2105,0.0026)
r_thigh_com_e =sb.Vector2(0.2274,-0.0026)
r_foot_com_e = sb.Vector2(0.0630,-0.0642)
box_com = sb.Vector2(0.3053,-0.2858)

# moment of inertia
calf_I_e = 0.0202
thigh_I_e = 0.0237
foot_I_e = 0.0086
box_I = (0.15**2*0.37**2)*m_backpack/12

#put parameters into array
l_e = [0,calf_l_e,thigh_l_e,0,thigh_l_e,calf_l_e]

rc_e = [f_calf_com_e,
         f_thigh_com_e,
         box_com,
         r_thigh_com_e,
         r_calf_com_e,
         r_foot_com_e]
m_e = [m_calf_e,m_thigh_e,m_backpack,m_thigh_e,m_calf_e,m_foot_e]

In [2]:
# generate forward kinematics

## define symbolic variable

q_e = sb.V6.symbolic('q')
dq_e = sb.V6.symbolic('dq')
# align the angles with exo's encoders
q_e[0] = 90+q_e[0] #front ankle = ank_ang+
q_e[3] = -180-q_e[3]
q_e[4] = -q_e[4]
q_e[5] = -90-q_e[5]

dq_e[3] = -dq_e[3]
dq_e[4] = -dq_e[4]
dq_e[5] = -dq_e[5]


q_e = q_e/180*np.pi
dq_e = dq_e/180*np.pi*100



In [3]:
# create transformation matrix function
q = sb.Symbol('q')
l = sb.Symbol('l')
T_mat = sb.Matrix([[sb.cos(q),-sb.sin(q)],
                   [sb.sin(q),sb.cos(q)]])


In [4]:
import pdb
#build rcm pos from joint angles
T = sb.Matrix([[1,0],
               [0,1]])
w = 0
P_exo=sb.Matrix.zeros(2,1)
v = sb.Vector2([0,0])
for i in range(6):
    T = (T*T_mat.subs({q:q_e[i]})).simplify()
    # pdb.set_trace()
    # rcm_pos = (T*sb.Vector3([rc_e[i][0],rc_e[i][1],1]))[0:2]
    w = w+dq_e[i]
    r_c = T*sb.Vector2([rc_e[i][0],rc_e[i][1]])
    r = T*sb.Vector2([l_e[i],0])
    vc = v+sb.Vector2([-w*r_c[1],w*r_c[0]])
    v = v+sb.Vector2([-w*r[1],w*r[0]])
    P_exo = P_exo+m_e[i]*vc
# T = T.simplify()
    

In [5]:
def ExoMomentum(q:sb.Vector6,dq:sb.Vector6)->sb.Matrix21:
    return P_exo

from symforce.codegen import Codegen, CppConfig
from symforce.notebook_util import display, display_code, display_code_file


codegen = Codegen.function(ExoMomentum, config=CppConfig())
Test_codegen_data = codegen.generate_function(output_dir='/home/lowlimb/cdrive/UCLA/lab/Exoskeleton/Controller/ExoRPI4/DynGen')
# display_code_file(Test_codegen_data.generated_files[0], "C++")